# Importo los paquetes

In [ ]:
import folium
import geopandas
import geocoder
import re
import matplotlib.pyplot as plt
from branca.element import Figure

# Archivos a utilizar

In [ ]:
file_accidentalidad="Accidentalidad_en_Barranquilla.csv"
file_semaforizacion="semaforizacion.csv"
file_fotodeteccion="Fotodetecci_n_en_Barranquilla.csv"

# Dataframes

In [ ]:
df_fotodeteccion=geopandas.read_file(file_fotodeteccion)
df_semaforizacion = geopandas.read_file(file_semaforizacion)
df_accidentalidad= geopandas.read_file(file_accidentalidad)

# Función (Extrae la hora de cada horario)

In [ ]:
def funcion(argumento):
    if re.search(".PM",argumento)!= None:
        cambio=12+int(argumento[:2])
    else:
        cambio=int(argumento[:2])
    return cambio

# Parametros variables

In [ ]:
inicio_mes=35040
final_mes=35307
mes="Abril"
anio="2021"

# Diccionario {Hora:Cantidad de accidentes}

In [ ]:
horarios=dict(df_accidentalidad["HORA_ACCIDENTE"].iloc[inicio_mes:final_mes].apply(lambda x: funcion(x)).value_counts(sort=False))

# Lo convierto en listas separadas

In [ ]:
lista_hora=[]
lista_accidentes=[]
for i,j in horarios.items():
    lista_hora.append(i)
    lista_accidentes.append(j)

# Distribución accidentes por Hora

In [ ]:
fig = plt.figure( figsize=[10,5])
ax = fig.add_axes([0,0,1,1])



ax.bar(lista_hora, lista_accidentes)

ax.set_xlabel('Hora',fontsize=15,color="red")
ax.set_title("Distribución Accidentes por hora ("+mes+")",fontsize=30,color="red")
plt.yticks(ticks=[*range(0,40,5)], labels=[ str(i) for i in [*range(0,40,5)]],fontsize=25)
plt.xticks(ticks=[*range(0,25)], labels=[ str(i) for i in [*range(0,25)]],fontsize=20)
plt.savefig("Numero_accidentes_"+mes+"_"+anio+".png")
plt.show()

# Nueva Columna con dirección de accidentes escrita adecuadamente

In [ ]:
df_accidentalidad["SITIO_EXACTO_ACCIDENTE_modificada"]=df_accidentalidad["SITIO_EXACTO_ACCIDENTE"].str.replace('CL','CALLE')
df_accidentalidad["SITIO_EXACTO_ACCIDENTE_modificada"]=df_accidentalidad["SITIO_EXACTO_ACCIDENTE_modificada"].str.replace("CALLELE","CALLE")
df_accidentalidad["SITIO_EXACTO_ACCIDENTE_modificada"]=df_accidentalidad["SITIO_EXACTO_ACCIDENTE_modificada"].str.replace("CR","CARRERA")
df_accidentalidad["SITIO_EXACTO_ACCIDENTE_modificada"]=df_accidentalidad["SITIO_EXACTO_ACCIDENTE_modificada"].str.replace("CARRERAA","CARRERA")


# Lista con Latitud y Longitud de la ubicación de cada accidente

In [ ]:
lista_coordenadas_accidentes=[]
for direccion in df_accidentalidad["SITIO_EXACTO_ACCIDENTE_modificada"].iloc[inicio_mes:final_mes]:
    try:
        lista_coordenadas_accidentes.append(geocoder.osm(direccion+",Barranquilla,Colombia").geojson['features'][0]["geometry"]["coordinates"])
    except:
        pass

# Cambio las ubicaciones del orden de los elementos en la lista (estaban al revez la latitud y la longitud)

In [ ]:
for i,j in enumerate(lista_coordenadas_accidentes):
    lista_coordenadas_accidentes[i]=[j[1],j[0]]

# Listas de ubicación (Latitud y Longitud)

In [ ]:
LATITUD_fotodeteccion=df_fotodeteccion["LATITUD"]
LONGITUD_fotodeteccion=df_fotodeteccion["LONGITUD"]
LATITUD_semaforizacion=df_semaforizacion["LATITUD"]
LONGITUD_semaforizacion=df_semaforizacion["LONGITUD"]

In [ ]:
lista_coordenadas_fotodeteccion=[]
for latitud_fotodeteccion,longitud_fotodeteccion in zip(LATITUD_fotodeteccion,LONGITUD_fotodeteccion):
    lista_coordenadas_fotodeteccion.append([latitud_fotodeteccion,longitud_fotodeteccion])

In [ ]:
lista_coordenadas_semaforizacion=[]
for latitud_semaforizacion,longitud_semaforizacion in zip(LATITUD_semaforizacion,LONGITUD_semaforizacion):
    lista_coordenadas_semaforizacion.append([latitud_semaforizacion,longitud_semaforizacion])

# Defino una figura por cada mapa

In [ ]:
fig_1=Figure(width=500,height=300)
fig_2=Figure(width=500,height=300)
fig_3=Figure(width=500,height=300)

# Distribucion de accidentes por mes

In [ ]:
m_1=folium.Map(location=[10.9878, -74.7889],zoom_start=11,min_zoom=8,max_zoom=14)

for index in range(len(lista_coordenadas_accidentes)):
    folium.Circle(location=lista_coordenadas_accidentes[index],
                  radius=70,
                  color='black').add_to(m_1)

fig_1.add_child(m_1)
m_1

# Distribucion semaforos en la ciudad

In [ ]:
m_2=folium.Map(location=[10.9878, -74.7889],zoom_start=11,min_zoom=8,max_zoom=14)

for index in range(len(lista_coordenadas_semaforizacion)):
    folium.Circle(location=lista_coordenadas_semaforizacion[index],
                  radius=1,
                  color='red').add_to(m_2)

fig_2.add_child(m_2)
m_2

# Distribución Fotodetectores en la Ciudad

In [ ]:
m_3=folium.Map(location=[10.9878, -74.7889],zoom_start=11,min_zoom=8,max_zoom=14)

for index in range(len(lista_coordenadas_fotodeteccion)):
    folium.Marker(location=lista_coordenadas_fotodeteccion[index],
                  radius=10,
                  color='black').add_to(m_3)

fig_3.add_child(m_3)
m_3